In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


In [ ]:
##### FInd News Articles ###
url = "https://redplanetscience.com/"
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
request = requests.get(url)
browser.visit(url)

In [ ]:
html = browser.html
soup = bs(html, "html.parser")
browser.quit()

In [ ]:
soup_list = soup.find_all("div", class_="list_text")

In [ ]:
headline = []
description = []

In [ ]:
for i in soup_list:
    headlines = i.find("div", "content_title").text
    descriptions = i.find("div", "article_teaser_body").text
    headline.append(headlines)
    description.append(descriptions)

In [ ]:
##### FInd Featured Photo ###
url = "https://spaceimages-mars.com/"
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
request = requests.get(url)
browser.visit(url)
time.sleep(1)

In [ ]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [ ]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
soup1 = soup.find("div", class_="floating_text_area")
photo_link = soup1.select_one("a",target_="_blank", class_="showimg fancybox-thumbs")["href"]
browser.quit()

In [ ]:
######## Find Mars Data #############
url = "https://galaxyfacts-mars.com/"
tables = pd.read_html(url)
tables

df = tables[0]
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

In [93]:
######## Find  #############
url = "https://marshemispheres.com/"
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
request = requests.get(url)
browser.visit(url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")
soupp = soup.find('div', class_='collapsible results')
items = soupp.find_all("div", class_="item")



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\steve\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [94]:
scrape_link = []
picture_description = []
#image links
for i in items:
    desc_container = i.find("a", class_="itemLink product-item")['href']
    scrape_link.append(desc_container)
    picture_description.append(i.find("h3").text)

In [97]:
image_link = []
for i in scrape_link:
    scrapable_url = f'{url}{i}'
    browser.visit(scrapable_url)
    # time.sleep(1)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find('div', class_='downloads')
    image_url = downloads.find("a")["href"]
    image_link.append(f'{url}{image_url}')

print(image_link)
    

['https://marshemispheres.com/images/full.jpg', 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg', 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg', 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg']
